In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'control': 0, '100nM': 1}

# Load graph dataset and process if neede
graph_path = data_dir / '13cyc_3D' / 'graphs' 

dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset2D3D(2492):
Number of graphs: 2492
Number of features: 13
Number of classes: 2
Train set: 1197, test set: 996, val set: 299
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 58354], pos=[4657, 2], labels=[4657, 13], nuclei=[4657], weight=[58354], condition=[32], fov=[32], id=[32], pos3D=[4657, 3], edge_attr=[58354, 1], x=[4657, 13], y=[32], edge_weight_2D=[58354], edge_weight_3D=[58354], name=[32], batch=[4657], ptr=[33])



In [9]:
# # Print some information on the dataset
# print(f'Dataset: {dataset}:')
# print('======================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
# for step, data in enumerate(train_loader):

#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()
#     break

In [10]:
data.edge_weight_2D

tensor([0.1474, 0.0617, 0.1485,  ..., 0.3465, 0.2191, 0.4646])

In [11]:
data.edge_weight_3D

tensor([3.6719e-03, 2.9284e-01, 3.3703e-03,  ..., 6.6989e-09, 1.0734e-08,
        2.2192e-21])

# K-fold on filter dataset

In [12]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [14]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 50

graph_path = data_dir / '13cyc_3D' / 'graphs' 
dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [15]:
len(dataset_filtered)

2153

In [16]:
dataset_name = '13PPI_2D3D_fusion'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / '13cyc_3D' /"saved_models_fusion" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [17]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)
epochs = 65

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# # model = 'GAT'
# model = 'GINConv'
params = [[2,64,'attention','MLP'], 
[2,16,'attention','GraphConv'],
[2,64,'attention2','GCN'],
[3,64,'attention','GAT'],
[3,16,'mean','GINConv']]

Global seed set to 42


In [18]:
params

[[2, 64, 'attention', 'MLP'],
 [2, 16, 'attention', 'GraphConv'],
 [2, 64, 'attention2', 'GCN'],
 [3, 64, 'attention', 'GAT'],
 [3, 16, 'mean', 'GINConv']]

In [19]:
import PPIGraph
import torch
torch.set_float32_matmul_precision('medium')

In [20]:
fusion = 'bilinear'

In [21]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fusion}_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}_{fusion}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold_fusion(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=13,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                batch_size=128,
                                                fusion=fusion,
                                                graph_pooling=pool)
        run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


Using MLP


c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | MLPModel         | 5.1 K 
2  | model3D     | MLPModel         | 5.1 K 
3  | head        | BilinearFusion   | 686 K 
4  | pred        | Sequential       | 130   
5  | loss_module | CrossEntropyLoss | 0     
6  | train_acc   | BinaryAccuracy   | 0     
7  | train_auroc | BinaryAUROC      | 0     
8  | train_f1    | BinaryF1Score    | 0     
9  | valid_acc   | BinaryAccuracy   | 0     
10 | valid_auroc | BinaryAUROC      | 0     
11 | valid_f1    | BinaryF1Score    | 0     
12 | pool2D      | GlobalAttention  | 65    
13 | pool3D      | GlobalAttention  | 65    
----------------

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▇▇▇▇▇▇▇▇█▇█▇█▇▇▇▇█▇▇█▇▇█████▇▇██████▇▇
train_auc,▁▃▇▇▇▇▇▇█████████▇█████████████▇████████
train_f1,▁▄▇▇█▇▇█████████████████████████████████
train_loss_epoch,█▇▃▂▂▂▃▂▂▁▂▁▁▁▂▁▂▂▂▁▂▁▁▁▂▁▁▁▁▁▂▂▁▁▁▁▁▁▂▂
train_loss_step,█▇▄▃▆▃▆▄▁▁▄▄▃▃▁▂▅▃▄▄▃▂▃▂▂▃▂▃▂▄▃▃▃▁▂▂▂▁▃▃
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▇█▇▆▇▇███████▇█▇███████████████████████
val_auc,▁▃▄▆▇▇▇▇▇▇████████████████████▇█▇███▇▇██
val_f1,▁▇█▇▇███████████████████████████████████


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▂▅▆▆▆▅▅▆▇▆▆▆▆▅▇▇▇▆▇▇▇▇▇▇▇▇▇██▇▇██▇▇███▇
train_auc,▁▂▆▆▆▆▆▆▇▇▇▇▇▇▆▇▇▇▇▇██████▇████▇████████
train_f1,▁▂▅▆▆▆▅▅▆▇▆▆▆▆▆▇▇▇▆▇▆▇▇▇▇▇█▇█▇▇▇██▇▇███▇
train_loss_epoch,█▇▄▃▄▃▃▄▃▂▃▂▃▃▃▂▂▂▃▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂▂▂▁▁▂▂
train_loss_step,▆▃▂▆▂▅▃▄▄▁▃▃▄▄▂▅▄▂█▃▁▃▄▂▃▂▃▄▃▂▄▂▂▃▃▄▃▃▂▃
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▁▅▇▇▇▆▇▇▇▇▇▇▇▇▅██▆█▇██▆▆███▇▇▇▇██▇█████
val_auc,▁▃▄▅▅▅▆▄▅▅▆▅▆▇▆▆▇▆▆▇▆▇▇▄▆▇▇▇▇▇▇▇▇█▇▇████
val_f1,▁▁▄▇▆▇▆▇▇▇▇▇▇▇▇▅▇▇▆▇▆▇▇▃▆▇▇█▇▇▅▄██▇█▇█▇█


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GIN              | 1.7 K 
2  | model3D     | GIN              | 1.7 K 
3  | head        | BilinearFusion   | 19.6 K
4  | pred        | Sequential       | 34    
5  | loss_module | CrossEntropyLoss | 0     
6  | train_acc   | BinaryAccuracy   | 0     
7  | train_auroc | BinaryAUROC      | 0     
8  | train_f1    | BinaryF1Score    | 0     
9  | valid_acc   | BinaryAccuracy   | 0     
10 | valid_auroc | BinaryAUROC      | 0     
11 | valid_f1    | BinaryF1Score    | 0     
--------------------------------------------------
23.2 K    Trainable params
0         Non-trainable params
23.2 K    Tota

GINConv


`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇██▇▇█▇███▇▇▇████████
train_auc,▁▆▆▇▇▇▇█▇▇▇▇█▇▇▇▇▇█████▇█▇████▇▇████████
train_f1,▁▆▇▇▇▇▇█▇▇▇▇▇▇▇████████▇█████▇██████████
train_loss_epoch,█▄▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁
train_loss_step,█▆▂▇▄▄▄▄▃▄▄▄▃▄▃▃▄▃▄▄▄▂▅▃▂▃▃▄▃▃▁▃▂▂▄▄▅▄▂▄
trainer/global_step,▁▁▁▁▁▁▁▁▁▃▃▃▃▃▄▂▂▂▂▂▂▅▅▅▅▆▆▂▂▂▂▃▃▇▇▇███▃
val_acc,▁▁▄▅▅▆▅▆▆▆▇█▆▆▇▇▇▇▆▅▆▆█▄▆▅▆▅▆▆▆▆▆▇▇▅▅█▆▆
val_auc,▁▁▆▅▇▇▇▆▇▇▇██▇▇▇▆█▇▇█▇█▅▆▇▇▇█▇▆▇▆▇▇▆▇▇▇▆
val_f1,▁▅▄▅▅▇▆▆▆▆▇█▇▆▇▇▇▇▆▅▆▇█▅▇▅▆▅▆▆▅▆▇▇▇▅▆▇▆▆
